In [1]:
import pandas as pd
import os
from pathlib import Path

# Đường dẫn gốc
BASE_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord")

# Định nghĩa các file cần gộp
FILES_CONFIG = {
    "Amazon": {
        "train": BASE_PATH / "Amazon" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Amazon" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Amazon" / "test_reviews_under80words.csv"
    },
    "Hotels": {
        "train": BASE_PATH / "Hotels" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Hotels" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Hotels" / "test_reviews_under80words.csv"
    },
    "AmazonsElectronics": {
        "all": BASE_PATH / "AmazonsElectronics" / "Kaggle" / "neutral_output_aspect_filledNone.csv"
    }
}

print("Cấu hình đường dẫn files:")
for source, files in FILES_CONFIG.items():
    print(f"\n{source}:")
    for file_type, path in files.items():
        exists = "✓" if path.exists() else "✗"
        print(f"  {file_type}: {path.name} [{exists}]")

Cấu hình đường dẫn files:

Amazon:
  train: train_reviews_under80words.csv [✓]
  dev: dev_reviews_under80words.csv [✓]
  test: test_reviews_under80words.csv [✓]

Hotels:
  train: train_reviews_under80words.csv [✓]
  dev: dev_reviews_under80words.csv [✓]
  test: test_reviews_under80words.csv [✓]

AmazonsElectronics:
  all: neutral_output_aspect_filledNone.csv [✓]


In [2]:
def load_and_standardize_data(file_path):
    """
    Đọc file CSV và chuẩn hóa cột thành reviewText, AspectTerm, Sentiment
    
    Parameters:
    - file_path: đường dẫn file
    
    Returns:
    - DataFrame đã chuẩn hóa với 3 cột: reviewText, AspectTerm, Sentiment
    """
    try:
        df = pd.read_csv(file_path)
        
        # Chuẩn hóa tên cột thành chữ thường để dễ xử lý
        df.columns = df.columns.str.strip().str.lower()
        
        # Map các tên cột có thể khác nhau về tên chuẩn
        column_mapping = {
            'reviewtext': 'reviewText',
            'review': 'reviewText',
            'text': 'reviewText',
            'aspectterm': 'AspectTerm',
            'aspect': 'AspectTerm',
            'sentiment': 'Sentiment'
        }
        
        df = df.rename(columns=column_mapping)
        
        # Chỉ giữ 3 cột cần thiết
        required_cols = ['reviewText', 'AspectTerm', 'Sentiment']
        available_cols = [col for col in required_cols if col in df.columns]
        df = df[available_cols]
        
        # Điền 'None' vào các AspectTerm trống hoặc null
        if 'AspectTerm' in df.columns:
            # Đếm số lượng trước khi điền
            empty_mask = df['AspectTerm'].isna() | (df['AspectTerm'].astype(str).str.strip() == '')
            empty_count = empty_mask.sum()
            
            # Điền 'None' vào các ô trống
            df.loc[empty_mask, 'AspectTerm'] = 'No Aspect'
            
            print(f"  ✓ Loaded {len(df)} rows from {file_path.name} (filled {empty_count} empty AspectTerm with 'None')")
        else:
            print(f"  ✓ Loaded {len(df)} rows from {file_path.name}")
        
        return df
        
    except Exception as e:
        print(f"  ✗ Error loading {file_path}: {e}")
        return pd.DataFrame()

print("Hàm load_and_standardize_data đã được định nghĩa!")

Hàm load_and_standardize_data đã được định nghĩa!


In [3]:
def merge_all_datasets(files_config, output_path=None, balance_config=None):
    """
    Gộp tất cả các dataset thành một file duy nhất
    
    Parameters:
    - files_config: dict chứa cấu hình các file
    - output_path: đường dẫn lưu file output (optional)
    - balance_config: dict để giới hạn số lượng mẫu theo sentiment (optional)
                      Ví dụ: {'positive': 5000, 'negative': 5000, 'neutral': 5000}
                      Nếu None, giữ nguyên tất cả dữ liệu
    
    Returns:
    - DataFrame đã gộp với 3 cột: reviewText, AspectTerm, Sentiment
    """
    all_dataframes = []
    
    print("=" * 60)
    print("BẮT ĐẦU GỘP DATASETS")
    print("=" * 60)
    
    for source_name, files in files_config.items():
        print(f"\n📂 Loading from {source_name}:")
        for file_type, file_path in files.items():
            if file_path.exists():
                df = load_and_standardize_data(file_path)
                if not df.empty:
                    all_dataframes.append(df)
            else:
                print(f"  ⚠ File not found: {file_path}")
    
    if not all_dataframes:
        print("\n❌ Không có dữ liệu nào được load!")
        return pd.DataFrame()
    
    # Gộp tất cả
    merged_df = pd.concat(all_dataframes, ignore_index=True)
    
    print("\n" + "=" * 60)
    print("KẾT QUẢ GỘP DATASET (TRƯỚC KHI CÂN BẰNG)")
    print("=" * 60)
    print(f"\n📊 Tổng số mẫu: {len(merged_df):,}")
    print(f"\n📈 Phân bố theo Sentiment:")
    print(merged_df['Sentiment'].value_counts().to_string())
    
    # Áp dụng balance_config nếu có
    if balance_config:
        print("\n" + "=" * 60)
        print("ÁP DỤNG CÂN BẰNG DỮ LIỆU")
        print("=" * 60)
        
        balanced_dfs = []
        for sentiment, max_count in balance_config.items():
            sentiment_df = merged_df[merged_df['Sentiment'].str.lower() == sentiment.lower()]
            current_count = len(sentiment_df)
            
            if current_count > max_count:
                sentiment_df = sentiment_df.sample(n=max_count, random_state=42)
                print(f"  {sentiment}: {current_count:,} → {max_count:,} (giảm {current_count - max_count:,})")
            else:
                print(f"  {sentiment}: {current_count:,} (giữ nguyên, ít hơn {max_count:,})")
            
            balanced_dfs.append(sentiment_df)
        
        merged_df = pd.concat(balanced_dfs, ignore_index=True)
        merged_df = merged_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print("\n" + "=" * 60)
    print("KẾT QUẢ CUỐI CÙNG")
    print("=" * 60)
    print(f"\n📊 Tổng số mẫu: {len(merged_df):,}")
    print(f"\n📈 Phân bố theo Sentiment:")
    print(merged_df['Sentiment'].value_counts().to_string())
    print(f"\n📋 Các cột: {list(merged_df.columns)}")
    
    # Lưu file nếu có output_path
    if output_path:
        merged_df.to_csv(output_path, index=False)
        print(f"\n💾 Đã lưu file: {output_path}")
    
    return merged_df

print("Hàm merge_all_datasets đã được định nghĩa!")

Hàm merge_all_datasets đã được định nghĩa!


In [4]:
# ============================================
# CẤU HÌNH SỐ LƯỢNG MẪU THEO SENTIMENT
# ============================================
# Chỉnh sửa các giá trị bên dưới để thay đổi số lượng mẫu cho mỗi nhãn
# Đặt None nếu muốn giữ tất cả dữ liệu không giới hạn

BALANCE_CONFIG = {
    'positive': 6600,   # Số lượng mẫu positive tối đa
    'negative': 6600,   # Số lượng mẫu negative tối đa  
    'neutral': 5000    # Số lượng mẫu neutral tối đa
}

# Đặt BALANCE_CONFIG = None nếu muốn giữ tất cả dữ liệu
# BALANCE_CONFIG = None

# ============================================
# Thực hiện gộp dataset
OUTPUT_PATH = BASE_PATH / "merged_dataset_all_13200rows.csv"

merged_df = merge_all_datasets(
    FILES_CONFIG, 
    output_path=OUTPUT_PATH,
    balance_config=BALANCE_CONFIG
)

BẮT ĐẦU GỘP DATASETS

📂 Loading from Amazon:
  ✓ Loaded 6297 rows from train_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 974 rows from dev_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 966 rows from test_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')

📂 Loading from Hotels:
  ✓ Loaded 10126 rows from train_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 1036 rows from dev_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')
  ✓ Loaded 242 rows from test_reviews_under80words.csv (filled 0 empty AspectTerm with 'None')

📂 Loading from AmazonsElectronics:
  ✓ Loaded 5000 rows from neutral_output_aspect_filledNone.csv (filled 3788 empty AspectTerm with 'None')

KẾT QUẢ GỘP DATASET (TRƯỚC KHI CÂN BẰNG)

📊 Tổng số mẫu: 24,641

📈 Phân bố theo Sentiment:
Sentiment
positive    15937
neutral      5576
negative     3116

ÁP DỤNG CÂN BẰNG DỮ LIỆU
  positive: 15,937 → 6,600 (giảm 

In [5]:
# Xem một số mẫu dữ liệu
print("📋 Một số mẫu dữ liệu đầu tiên:")
merged_df.head(10)

📋 Một số mẫu dữ liệu đầu tiên:


,reviewText,AspectTerm,Sentiment
0,I have seen and purchased these at lower price...,No Aspect,neutral
1,"I was thinking about other thing, it was very ...",No Aspect,neutral
2,"There are Better Options than this tired , out...",hotel quality,negative
3,"I think is good, but I think creative labs pro...",No Aspect,neutral
4,"This is NOT an Apple product. Although, it is ...",No Aspect,neutral
5,"Basically , you end up paying well over 2 time...",food prices,negative
6,"There ' s no bitter after taste , and it ' s g...",food quality,positive
7,"I like the fact that there are no additives , ...",food quality,positive
8,Restaurant very odd .,facilities general,negative
9,Appears to do the trick for being able to char...,price,neutral


## Tổng kết

File đã được tạo với cấu trúc 3 cột:
- `reviewText`: Nội dung review
- `AspectTerm`: Aspect term được trích xuất
- `Sentiment`: Nhãn sentiment (positive/negative/neutral)

In [8]:
# ============================================
# GỘP 2 DATASET: merged_dataset + semeval_train
# ============================================
print("=" * 60)
print("GỘP DATASET: merged_dataset + semeval_train")
print("=" * 60)

# Đường dẫn các file
MERGED_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord/merged_dataset_all_13200rows.csv")
SEMEVAL_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/Semeval/semeval_train.csv")

# Đọc 2 file
df_merged = pd.read_csv(MERGED_PATH)
df_semeval = pd.read_csv(SEMEVAL_PATH)

print(f"\n📂 Dataset 1: merged_dataset_all_13200rows.csv")
print(f"   Số dòng: {len(df_merged):,}")
print(f"   Các cột: {list(df_merged.columns)}")
print(f"   Phân bố Sentiment:")
print(df_merged['Sentiment'].value_counts().to_string())

print(f"\n📂 Dataset 2: semeval_train.csv")
print(f"   Số dòng: {len(df_semeval):,}")
print(f"   Các cột: {list(df_semeval.columns)}")
print(f"   Phân bố Sentiment:")
print(df_semeval['Sentiment'].value_counts().to_string())

GỘP DATASET: merged_dataset + semeval_train

📂 Dataset 1: merged_dataset_all_13200rows.csv
   Số dòng: 14,716
   Các cột: ['reviewText', 'AspectTerm', 'Sentiment']
   Phân bố Sentiment:
Sentiment
positive    6600
neutral     5000
negative    3116

📂 Dataset 2: semeval_train.csv
   Số dòng: 5,416
   Các cột: ['reviewText', 'Sentiment', 'AspectTerm']
   Phân bố Sentiment:
Sentiment
positive    3294
negative    1833
neutral      289


In [9]:
# Chuẩn hóa tên cột của semeval_train để khớp với merged_dataset
# semeval_train có: reviewText, Sentiment, AspectTerms
# merged_dataset có: reviewText, AspectTerm, Sentiment

# Rename cột AspectTerms -> AspectTerm (bỏ s)
if 'AspectTerms' in df_semeval.columns:
    df_semeval = df_semeval.rename(columns={'AspectTerms': 'AspectTerm'})
    print("✓ Đã đổi tên cột 'AspectTerms' -> 'AspectTerm'")

# Sắp xếp lại thứ tự cột cho khớp
df_semeval = df_semeval[['reviewText', 'AspectTerm', 'Sentiment']]

# Gộp 2 dataset
df_combined = pd.concat([df_merged, df_semeval], ignore_index=True)

# Shuffle dữ liệu
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

print("\n" + "=" * 60)
print("KẾT QUẢ GỘP 2 DATASET")
print("=" * 60)
print(f"\n📊 Tổng số mẫu: {len(df_combined):,}")
print(f"   - Từ merged_dataset: {len(df_merged):,}")
print(f"   - Từ semeval_train: {len(df_semeval):,}")

print(f"\n📈 Phân bố Sentiment sau khi gộp:")
print(df_combined['Sentiment'].value_counts().to_string())

print(f"\n📋 Các cột: {list(df_combined.columns)}")


KẾT QUẢ GỘP 2 DATASET

📊 Tổng số mẫu: 20,132
   - Từ merged_dataset: 14,716
   - Từ semeval_train: 5,416

📈 Phân bố Sentiment sau khi gộp:
Sentiment
positive    9894
neutral     5289
negative    4949

📋 Các cột: ['reviewText', 'AspectTerm', 'Sentiment']


In [10]:
# ============================================
# CÂN BẰNG DỮ LIỆU (TÙY CHỌN)
# ============================================
# Chỉnh sửa config bên dưới nếu muốn cân bằng

BALANCE_COMBINED = {
    'positive': 10000,   # Số lượng mẫu positive tối đa
    'negative': 10000,   # Số lượng mẫu negative tối đa  
    'neutral': 5000      # Số lượng mẫu neutral tối đa
}

# Đặt BALANCE_COMBINED = None nếu không muốn cân bằng
# BALANCE_COMBINED = None

if BALANCE_COMBINED:
    print("=" * 60)
    print("ÁP DỤNG CÂN BẰNG DỮ LIỆU")
    print("=" * 60)
    
    balanced_dfs = []
    for sentiment, max_count in BALANCE_COMBINED.items():
        sentiment_df = df_combined[df_combined['Sentiment'].str.lower() == sentiment.lower()]
        current_count = len(sentiment_df)
        
        if current_count > max_count:
            sentiment_df = sentiment_df.sample(n=max_count, random_state=42)
            print(f"  {sentiment}: {current_count:,} → {max_count:,} (giảm {current_count - max_count:,})")
        else:
            print(f"  {sentiment}: {current_count:,} (giữ nguyên)")
        
        balanced_dfs.append(sentiment_df)
    
    df_final = pd.concat(balanced_dfs, ignore_index=True)
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
else:
    df_final = df_combined

print(f"\n📊 Tổng số mẫu cuối cùng: {len(df_final):,}")
print(f"\n📈 Phân bố Sentiment cuối cùng:")
print(df_final['Sentiment'].value_counts().to_string())

ÁP DỤNG CÂN BẰNG DỮ LIỆU
  positive: 9,894 (giữ nguyên)
  negative: 4,949 (giữ nguyên)
  neutral: 5,289 → 5,000 (giảm 289)

📊 Tổng số mẫu cuối cùng: 19,843

📈 Phân bố Sentiment cuối cùng:
Sentiment
positive    9894
neutral     5000
negative    4949


In [11]:
# Lưu file gộp
OUTPUT_COMBINED_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord/merged_dataset_combined_semeval.csv")
df_final.to_csv(OUTPUT_COMBINED_PATH, index=False)

print(f"✅ Đã lưu file gộp: {OUTPUT_COMBINED_PATH}")
print(f"   Tổng số dòng: {len(df_final):,}")

# Xem mẫu dữ liệu
print("\n📋 Mẫu dữ liệu (10 dòng đầu):")
df_final.head(10)

✅ Đã lưu file gộp: H:\SentimentAnalystSchool\MidtermExam\DatasetUnder80kWord\merged_dataset_combined_semeval.csv
   Tổng số dòng: 19,843

📋 Mẫu dữ liệu (10 dòng đầu):


,reviewText,AspectTerm,Sentiment
0,That made me really sad because I really wante...,food general,negative
1,The friction pad is good to use in your car. ...,No Aspect,neutral
2,I wasn ' t sure how I would like the smoked ho...,food general,positive
3,I ordered these thinking they were something d...,No Aspect,neutral
4,"There are a ton of hotels and I say , look els...",hotel miscellaneous,negative
5,"Key Word = Budget Monte Carlo is clean , fairl...",location general,positive
6,"The ambience was nice, but service wasn't so g...",SERVICE-GENERAL,negative
7,Excellent value ( at London prices ) for a lux...,hotel prices,positive
8,We enjoyed eating at Ginger and Olsen - two ne...,service general,positive
9,"Room was very comfortable and modern , and goo...",rooms design_features,positive


# Gộp merged_dataset_combined_semeval.csv + df_train_all_new.csv

Gộp 2 file dataset với định dạng chuẩn `reviewText, AspectTerm, Sentiment`

In [14]:
import pandas as pd
import os
from pathlib import Path

def standardize_columns(df):
    """
    Chuẩn hóa tên cột của DataFrame thành format chuẩn: reviewText, AspectTerm, Sentiment
    
    Hỗ trợ các format input:
    - Format 1: reviewText, AspectTerm, Sentiment (đã chuẩn)
    - Format 2: text, aspect, polarity (SemEval format)
    - Format 3: domain, review_id, sentence_id, text, aspect, polarity, label
    
    Parameters:
    - df: DataFrame cần chuẩn hóa
    
    Returns:
    - DataFrame đã chuẩn hóa với 3 cột: reviewText, AspectTerm, Sentiment
    """
    # Tạo copy để không ảnh hưởng df gốc
    df = df.copy()
    
    # Chuẩn hóa tên cột thành chữ thường để dễ xử lý
    df.columns = df.columns.str.strip().str.lower()
    
    # Map các tên cột có thể khác nhau về tên chuẩn
    column_mapping = {
        # reviewText
        'reviewtext': 'reviewText',
        'review': 'reviewText',
        'text': 'reviewText',
        'sentence': 'reviewText',
        'content': 'reviewText',
        # AspectTerm  
        'aspectterm': 'AspectTerm',
        'aspectterms': 'AspectTerm',
        'aspect': 'AspectTerm',
        'aspect_term': 'AspectTerm',
        # Sentiment
        'sentiment': 'Sentiment',
        'polarity': 'Sentiment',
        'label': 'Sentiment'  # Nếu không có polarity thì dùng label
    }
    
    # Ưu tiên polarity hơn label nếu cả 2 đều có
    if 'polarity' in df.columns and 'label' in df.columns:
        # Giữ polarity, bỏ qua label trong mapping
        df = df.rename(columns={'polarity': 'Sentiment', 'text': 'reviewText', 'aspect': 'AspectTerm'})
    else:
        df = df.rename(columns=column_mapping)
    
    # Chỉ giữ 3 cột cần thiết
    required_cols = ['reviewText', 'AspectTerm', 'Sentiment']
    available_cols = [col for col in required_cols if col in df.columns]
    
    if len(available_cols) < 3:
        missing = set(required_cols) - set(available_cols)
        print(f"  ⚠ Thiếu các cột: {missing}")
        print(f"  📋 Các cột hiện có: {list(df.columns)}")
    
    df = df[available_cols]
    
    # Điền 'No Aspect' vào các AspectTerm trống hoặc null
    if 'AspectTerm' in df.columns:
        empty_mask = df['AspectTerm'].isna() | (df['AspectTerm'].astype(str).str.strip() == '')
        empty_count = empty_mask.sum()
        df.loc[empty_mask, 'AspectTerm'] = 'No Aspect'
        if empty_count > 0:
            print(f"  ✓ Đã điền {empty_count} AspectTerm trống với 'No Aspect'")
        
        # Chuyển # thành - trong AspectTerm (LAPTOP#GENERAL -> LAPTOP-GENERAL)
        df['AspectTerm'] = df['AspectTerm'].astype(str).str.replace('#', '-', regex=False)
    
    # Chuẩn hóa Sentiment về lowercase
    if 'Sentiment' in df.columns:
        df['Sentiment'] = df['Sentiment'].astype(str).str.strip().str.lower()
    
    return df

print("✓ Hàm standardize_columns() đã được định nghĩa!")
print("  Hỗ trợ các format:")
print("  - reviewText, AspectTerm, Sentiment")
print("  - text, aspect, polarity (SemEval)")
print("  - domain, review_id, sentence_id, text, aspect, polarity, label")

✓ Hàm standardize_columns() đã được định nghĩa!
  Hỗ trợ các format:
  - reviewText, AspectTerm, Sentiment
  - text, aspect, polarity (SemEval)
  - domain, review_id, sentence_id, text, aspect, polarity, label


In [15]:
# ============================================
# GỘP: merged_dataset_combined_semeval + df_train_all_new
# ============================================
print("=" * 60)
print("GỘP DATASET: merged_dataset_combined_semeval + df_train_all_new")
print("=" * 60)

# Đường dẫn các file
MERGED_COMBINED_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord/merged_dataset_all_NoAspect_version.csv")
SEMEVAL_NEW_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/Semeval/df_train_all_new.csv")

# Đọc 2 file
df_merged_combined = pd.read_csv(MERGED_COMBINED_PATH)
df_semeval_new = pd.read_csv(SEMEVAL_NEW_PATH)

# Chuẩn hóa cột cho cả 2 dataset
print("\n📂 Dataset 1: merged_dataset_all_NoAspect_version.csv")
print(f"   Cột ban đầu: {list(df_merged_combined.columns)}")
df_merged_combined = standardize_columns(df_merged_combined)
print(f"   Số dòng: {len(df_merged_combined):,}")
print(f"   Cột sau chuẩn hóa: {list(df_merged_combined.columns)}")
print(f"   Phân bố Sentiment:")
print(df_merged_combined['Sentiment'].value_counts().to_string())

print(f"\n📂 Dataset 2: df_train_all_new.csv")
print(f"   Cột ban đầu: {list(df_semeval_new.columns)}")
df_semeval_new = standardize_columns(df_semeval_new)
print(f"   Số dòng: {len(df_semeval_new):,}")
print(f"   Cột sau chuẩn hóa: {list(df_semeval_new.columns)}")
print(f"   Phân bố Sentiment:")
print(df_semeval_new['Sentiment'].value_counts().to_string())

GỘP DATASET: merged_dataset_combined_semeval + df_train_all_new

📂 Dataset 1: merged_dataset_all_NoAspect_version.csv
   Cột ban đầu: ['reviewText', 'AspectTerm', 'Sentiment']
   Số dòng: 14,716
   Cột sau chuẩn hóa: ['reviewText', 'AspectTerm', 'Sentiment']
   Phân bố Sentiment:
Sentiment
positive    6600
neutral     5000
negative    3116

📂 Dataset 2: df_train_all_new.csv
   Cột ban đầu: ['domain', 'review_id', 'sentence_id', 'text', 'aspect', 'polarity', 'label']
   Số dòng: 5,705
   Cột sau chuẩn hóa: ['reviewText', 'AspectTerm', 'Sentiment']
   Phân bố Sentiment:
Sentiment
positive    3294
negative    1833
neutral      578


In [16]:
# Gộp 2 dataset
# Cả 2 file đều có cùng format: reviewText, AspectTerm, Sentiment
df_all_combined = pd.concat([df_merged_combined, df_semeval_new], ignore_index=True)

# Shuffle dữ liệu
df_all_combined = df_all_combined.sample(frac=1, random_state=42).reset_index(drop=True)

print("\n" + "=" * 60)
print("KẾT QUẢ GỘP 2 DATASET")
print("=" * 60)
print(f"\n📊 Tổng số mẫu: {len(df_all_combined):,}")
print(f"   - Từ merged_dataset_combined_semeval: {len(df_merged_combined):,}")
print(f"   - Từ semeval_train_new: {len(df_semeval_new):,}")

print(f"\n📈 Phân bố Sentiment sau khi gộp:")
print(df_all_combined['Sentiment'].value_counts().to_string())

print(f"\n📋 Các cột: {list(df_all_combined.columns)}")


KẾT QUẢ GỘP 2 DATASET

📊 Tổng số mẫu: 20,421
   - Từ merged_dataset_combined_semeval: 14,716
   - Từ semeval_train_new: 5,705

📈 Phân bố Sentiment sau khi gộp:
Sentiment
positive    9894
neutral     5578
negative    4949

📋 Các cột: ['reviewText', 'AspectTerm', 'Sentiment']


In [17]:
# Lưu file gộp cuối cùng
OUTPUT_ALL_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord/merged_all_final.csv")
df_all_combined.to_csv(OUTPUT_ALL_PATH, index=False)

print(f"✅ Đã lưu file gộp: {OUTPUT_ALL_PATH}")
print(f"   Tổng số dòng: {len(df_all_combined):,}")

# Xem mẫu dữ liệu
print("\n📋 Mẫu dữ liệu (10 dòng đầu):")
df_all_combined.head(10)

✅ Đã lưu file gộp: H:\SentimentAnalystSchool\MidtermExam\DatasetUnder80kWord\merged_all_final.csv
   Tổng số dòng: 20,421

📋 Mẫu dữ liệu (10 dòng đầu):


,reviewText,AspectTerm,Sentiment
0,"Bigger screen than expected, laptop with a des...",DISPLAY-DESIGN_FEATURES,positive
1,The hotel staff when we stayed there were extr...,hotel design_features,negative
2,The main tourist spots are within a twenty min...,location general,positive
3,Never buy from Lake Champlain .,food recommendation,negative
4,It had most of the features and all of the pow...,LAPTOP-OPERATION_PERFORMANCE,positive
5,I complained that this did not constitute an o...,service general,negative
6,Took it back as it was defective.,LAPTOP-QUALITY,negative
7,Even better than the photos ! In addition to a...,hotel design_features,positive
8,Works fine for charging (no data connection). ...,No Aspect,neutral
9,I bought them because of Hungry Girl ' s glowi...,food general,negative
